In [ ]:
import xarray as xr, colorcet
from colorcet import bmw, coolwarm

from xpublish.routers import xyz_router, base_router
from xpublish.utils.ows import validate_dataset

In [ ]:
ds = xr.tutorial.open_dataset(
    "air_temperature", chunks=dict(lat=5, lon=5),
)
ds

XYZ router accepts only spatial dimensions that are named 'x' and  'y'

In [ ]:
ds = ds.rename({"lat": "y", "lon": "x"})

In [ ]:
validate_dataset(ds)

XYZ uses morecantile to tile the dataarray. Morecantile supports different grids, not only the Web Mercator which is the most common and it is used by Google Maps and OpenStreetMap.
In this example the dataset needs to be reprojected to match the Web Mercator projection.

In [ ]:
ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
ds.rio.write_crs(4326, inplace=True)

ds = ds.rio.reproject(
    # epsg: 3857
    dst_crs= "+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs" 
    )

XYZ router accepts 2 map options to:
- crs_epsg: set the correct grid tiling 
- color_mapping: produce custom images setting the datashader parameters or matplotlib parameters

In [ ]:
crs_epsg =  3857

color_mapping = {
    
    "datashader_settings":{
            # parameters for datashader.Canvas.raster method
            "raster": {"upsample_method": "linear"}, 
            # parameters for datashader.transfer_functions.shade
            "shade": {  "cmap": colorcet.bmy,#["blue","red"], 
                        "how": "linear", 
                        "span": [float(ds["air"].min()), float(ds["air"].max())], 
                        "alpha": 200 }
                        }, 
}

xyz_router.set_options(crs_epsg, color_mapping=color_mapping)

In [ ]:
ds.rest(routers=[xyz_router, base_router])

In [ ]:
import nest_asyncio 
nest_asyncio.apply()

In [ ]:
ds.rest.serve(port=9000,log_level='info')